In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from joblib import dump, load
import os
import pickle
from joblib import load
import torch.nn as nn
from sklearn.base import BaseEstimator
import torch
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from matplotlib.lines import Line2D
import math 

traindata = pd.read_csv('/home/code/kzm/Recon_Model-main/data/test.csv')
unique_layers = traindata['Layer'].unique()
targ_list = ['DTC']  
layer_feature = traindata['Layer']

onehot_encoder = OneHotEncoder(categories=[unique_layers], handle_unknown='ignore')

for targ in targ_list:
    print(f"Predicting for target variable: {targ}")
    all_features = ['DEPTH', 'GR', 'RD', 'RS', 'DEN', 'DTC']
    feature_names = [f for f in all_features if f != targ]
    target = traindata[targ]
    features = traindata[feature_names].values
    features = np.nan_to_num(features)
    target = np.nan_to_num(target)

    # Load saved model
    predictor = load(f'/home/code/kzm/Recon_Model-main/model/model_{targ}.pkl')
    scalerX = load(f'/home/code/kzm/Recon_Model-main/model/feature_scaler_{targ}.pkl')
    scalerY = load(f'/home/code/kzm/Recon_Model-main/model/target_scaler_{targ}.pkl')
    onehot_encoder = load(f'/home/code/kzm/Recon_Model-main/model/onehot_encoder_{targ}.pkl')

    # Use the loaded onehotencoder to encode the layer column independently
    layer_onehot = onehot_encoder.transform(layer_feature.values.reshape(-1, 1)).toarray() 
    
    # Combine the layer feature with other features
    features_encoded = np.concatenate((features, layer_onehot), axis=1)
    
    # Standardize features
    features_scaled = scalerX.transform(features_encoded)

    # Prediction using loaded models
    predictor.eval()
    with torch.no_grad():
        features_scaled_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)
        target_pred_scaled = predictor(features_scaled_tensor)
        target_pred_scaled = target_pred_scaled.cpu().numpy()
    
    # Reverse scale predictor
    target_pred = scalerY.inverse_transform(target_pred_scaled.reshape(-1, 1)).flatten()

    test_rmse_original_scale = np.sqrt(mean_squared_error(target, target_pred))
    test_mae_original_scale = mean_absolute_error(target, target_pred)
    test_r2_original_scale = r2_score(target, target_pred)

   
    